In [2]:
import requests
import pandas as pd
import requests
import os
import json
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

API_URL = "https://www.ebi.ac.uk/ols4/api"
API_KEY = os.getenv('OPENAI_API_KEY')

if not API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

def embed_text(text):
    res = requests.post(
        'https://api.openai.com/v1/embeddings',
        headers={'Authorization': f'Bearer {API_KEY}'},
        json={'input': text, 'model': 'text-embedding-3-small'}
    )
    return res.json()['data'][0]['embedding']

def search_ols(vec):
    res = requests.post(f"{API_URL}/v2/classes/llm_embedding", data=json.dumps(vec), headers={
        'Content-Type': 'application/json'})
    return res.json()['elements']

vec = embed_text("diabetes mellitus")
res = search_ols(vec)

df = pd.DataFrame(res)
df = df[['ontologyId', 'curie', 'label', 'score']]
df = df[df['ontologyId'] == 'mondo']

print(df.to_markdown(index=False))

| ontologyId   | curie         | label                               |    score |
|:-------------|:--------------|:------------------------------------|---------:|
| mondo        | MONDO:0005015 | ['diabetes mellitus']               | 0.845933 |
| mondo        | MONDO:1010574 | ['diabetes mellitus, dog']          | 0.821607 |
| mondo        | MONDO:1010577 | ['diabetes mellitus, pig']          | 0.809479 |
| mondo        | MONDO:1010576 | ['diabetes mellitus, horse']        | 0.805619 |
| mondo        | MONDO:1010575 | ['diabetes mellitus, domestic cat'] | 0.803764 |
